In [3]:
import torch
from mmcv import Config

### model load

In [4]:
MODEL_PATH ='./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_1x_coco.py'
MODEL = 'cascade_rcnn_x101_64x4d_fpn_1x_coco'
NAME = 'multiscale'
WORK_DIR = NAME+'_'+MODEL

# config file 들고오기
cfg = Config.fromfile(MODEL_PATH)

In [5]:
def print_keys(arg_dict):
    print(list(arg_dict.keys()))

print_keys(cfg)

['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow']


### Dataset config

In [6]:
dataset_type = 'CocoDataset'
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
num_classes = len(classes)

root='../dataset/'

data = dict(
    samples_per_gpu=4,
    workers_per_gpu=4,
    train=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file=root+'kfold/cv_train1.json',
        img_prefix=root),
    val=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file=root+'kfold/cv_val1.json',
        img_prefix=root),
    test=dict(
        type=dataset_type,
        # explicitly add your class names to the field `classes`
        classes=classes,
        ann_file=root+'test.json',
        img_prefix=root))

### dataset pipeline config
- augmentation

In [7]:
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1024, 1024), (512, 512), (256, 256)],
        multiscale_mode="value",
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=[(1024,1024),(512,512),(256,256)],
        flip=True,
        transforms=[
            dict(
                type='Resize',
                keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


data['train']['pipeline'] = train_pipeline
data['val']['pipeline'] = test_pipeline
data['test']['pipeline'] = test_pipeline

In [8]:
cfg.data = data
cfg.data.test.test_mode = True
cfg.data.val.test_mode = True

### model config

In [9]:
for bbox_head in cfg.model.roi_head.bbox_head:
    bbox_head.num_classes = num_classes

# multi anchor box scale
cfg.model.rpn_head.anchor_generator.scales =[2,4,8,16]

### optimizer config

In [10]:
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
print(cfg.optimizer)
print(cfg.optimizer_config)

{'type': 'SGD', 'lr': 0.02, 'momentum': 0.9, 'weight_decay': 0.0001}
{'grad_clip': {'max_norm': 35, 'norm_type': 2}}


### other config

In [11]:
cfg.checkpoint_config = dict(interval=1)

In [12]:
cfg.model.test_cfg.rpn.nms.type = 'soft_nms'
#cfg.evaluation.metric = 'mAP'

In [13]:
cfg.seed = 1010
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{WORK_DIR}'

In [14]:
# wandb
log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(
                project='p-stage_object_detection',
                name=f'{WORK_DIR}'))])
cfg.log_config = log_config

### config workflow

In [15]:
#cfg.workflow = [('train',1),('val',1)]

## train

In [16]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

In [18]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!


/opt/ml/detection/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


In [19]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-09-29 05:17:20,661 - mmdet - INFO - Start running, host: root@2f4df6ced18c, work_dir: /opt/ml/detection/mmdetection/work_dirs/multiscale_cascade_rcnn_x101_64x4d_fpn_1x_coco
2021-09-29 05:17:20,662 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook  

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


wandb: Currently logged in as: larcane (use `wandb login --relogin` to force relogin)


/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/opt/ml/detection/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
2021-09-29 05:17:47,081 - mmdet - INFO - Epoch [1][10/980]	lr: 3.796e-04, eta: 6:28:57, time: 1.986, data_time: 0.248, memory: 18076, loss_rpn_cls: 0.7005, loss_rpn_bbox: 0.1331, s0.loss_cls: 1.9675, s0.acc: 63.0176, s0.loss_bbox: 0.0034, s1.loss_cls: 1.1512, s1.acc: 38.7109, s1.loss_bbox: 0.0007, s2.loss_cls: 0.5955, s2.acc: 17.5781, s2.loss_bbox: 0.0000, loss: 4.5519, grad_norm: 16.4402
2021-09-29 05:18:01,776 - mmdet - INFO - Epoch [1][20/980]	lr: 7.792e-04, eta: 5:38:05, time: 1.470, data_time: 0.011, memory: 18076, loss_rpn_cl

2021-09-29 05:22:47,012 - mmdet - INFO - Epoch [1][210/980]	lr: 8.372e-03, eta: 4:53:08, time: 1.608, data_time: 0.011, memory: 18672, loss_rpn_cls: 0.3209, loss_rpn_bbox: 0.1429, s0.loss_cls: 0.3518, s0.acc: 94.4276, s0.loss_bbox: 0.1303, s1.loss_cls: 0.0883, s1.acc: 97.4901, s1.loss_bbox: 0.0333, s2.loss_cls: 0.0285, s2.acc: 98.5328, s2.loss_bbox: 0.0047, loss: 1.1008, grad_norm: 1.9996
2021-09-29 05:23:03,184 - mmdet - INFO - Epoch [1][220/980]	lr: 8.771e-03, eta: 4:53:42, time: 1.617, data_time: 0.011, memory: 18672, loss_rpn_cls: 0.2780, loss_rpn_bbox: 0.1415, s0.loss_cls: 0.3512, s0.acc: 93.7587, s0.loss_bbox: 0.1374, s1.loss_cls: 0.1034, s1.acc: 97.0044, s1.loss_bbox: 0.0420, s2.loss_cls: 0.0337, s2.acc: 98.2905, s2.loss_bbox: 0.0073, loss: 1.0945, grad_norm: 2.0684
2021-09-29 05:23:17,214 - mmdet - INFO - Epoch [1][230/980]	lr: 9.171e-03, eta: 4:52:25, time: 1.403, data_time: 0.011, memory: 18672, loss_rpn_cls: 0.2819, loss_rpn_bbox: 0.1574, s0.loss_cls: 0.3605, s0.acc: 93.3299

...

2021-09-29 08:28:09,328 - mmdet - INFO - Epoch [6][430/980]	lr: 2.000e-02, eta: 2:42:54, time: 1.628, data_time: 0.010, memory: 19497, loss_rpn_cls: 0.1407, loss_rpn_bbox: 0.0785, s0.loss_cls: 0.3542, s0.acc: 91.4141, s0.loss_bbox: 0.1394, s1.loss_cls: 0.1671, s1.acc: 91.5555, s1.loss_bbox: 0.1178, s2.loss_cls: 0.0687, s2.acc: 92.9246, s2.loss_bbox: 0.0556, loss: 1.1219, grad_norm: 1.4624
2021-09-29 08:28:24,449 - mmdet - INFO - Epoch [6][440/980]	lr: 2.000e-02, eta: 2:42:39, time: 1.512, data_time: 0.010, memory: 19497, loss_rpn_cls: 0.1816, loss_rpn_bbox: 0.1114, s0.loss_cls: 0.3598, s0.acc: 92.0801, s0.loss_bbox: 0.1220, s1.loss_cls: 0.1662, s1.acc: 91.9531, s1.loss_bbox: 0.0947, s2.loss_cls: 0.0694, s2.acc: 93.1641, s2.loss_bbox: 0.0447, loss: 1.1500, grad_norm: 1.3779
2021-09-29 08:28:39,565 - mmdet - INFO - Epoch [6][450/980]	lr: 2.000e-02, eta: 2:42:23, time: 1.512, data_time: 0.011, memory: 19497, loss_rpn_cls: 0.2049, loss_rpn_bbox: 0.1049, s0.loss_cls: 0.3689, s0.acc: 90.8496

2021-09-29 08:33:28,256 - mmdet - INFO - Epoch [6][640/980]	lr: 2.000e-02, eta: 2:37:34, time: 1.417, data_time: 0.011, memory: 19497, loss_rpn_cls: 0.1661, loss_rpn_bbox: 0.1022, s0.loss_cls: 0.4264, s0.acc: 89.9753, s0.loss_bbox: 0.1557, s1.loss_cls: 0.2027, s1.acc: 90.2561, s1.loss_bbox: 0.1343, s2.loss_cls: 0.0810, s2.acc: 92.0938, s2.loss_bbox: 0.0609, loss: 1.3292, grad_norm: 1.5557
2021-09-29 08:33:42,290 - mmdet - INFO - Epoch [6][650/980]	lr: 2.000e-02, eta: 2:37:18, time: 1.403, data_time: 0.010, memory: 19497, loss_rpn_cls: 0.1652, loss_rpn_bbox: 0.1059, s0.loss_cls: 0.4319, s0.acc: 89.0332, s0.loss_bbox: 0.1893, s1.loss_cls: 0.1913, s1.acc: 89.9858, s1.loss_bbox: 0.1533, s2.loss_cls: 0.0726, s2.acc: 92.9360, s2.loss_bbox: 0.0637, loss: 1.3732, grad_norm: 1.5999
2021-09-29 08:33:57,531 - mmdet - INFO - Epoch [6][660/980]	lr: 2.000e-02, eta: 2:37:03, time: 1.524, data_time: 0.010, memory: 19497, loss_rpn_cls: 0.1984, loss_rpn_bbox: 0.1335, s0.loss_cls: 0.4794, s0.acc: 88.5791

2021-09-29 08:38:48,083 - mmdet - INFO - Epoch [6][850/980]	lr: 2.000e-02, eta: 2:32:16, time: 1.642, data_time: 0.011, memory: 19497, loss_rpn_cls: 0.2087, loss_rpn_bbox: 0.0893, s0.loss_cls: 0.4128, s0.acc: 89.7900, s0.loss_bbox: 0.1610, s1.loss_cls: 0.1879, s1.acc: 90.6738, s1.loss_bbox: 0.1183, s2.loss_cls: 0.0779, s2.acc: 91.4404, s2.loss_bbox: 0.0576, loss: 1.3135, grad_norm: 1.4138
2021-09-29 08:39:04,354 - mmdet - INFO - Epoch [6][860/980]	lr: 2.000e-02, eta: 2:32:02, time: 1.627, data_time: 0.011, memory: 19497, loss_rpn_cls: 0.1614, loss_rpn_bbox: 0.0974, s0.loss_cls: 0.3294, s0.acc: 91.8262, s0.loss_bbox: 0.1115, s1.loss_cls: 0.1623, s1.acc: 90.7758, s1.loss_bbox: 0.0995, s2.loss_cls: 0.0711, s2.acc: 91.9750, s2.loss_bbox: 0.0469, loss: 1.0795, grad_norm: 1.3268
2021-09-29 08:39:18,356 - mmdet - INFO - Epoch [6][870/980]	lr: 2.000e-02, eta: 2:31:45, time: 1.400, data_time: 0.010, memory: 19497, loss_rpn_cls: 0.1865, loss_rpn_bbox: 0.0811, s0.loss_cls: 0.4017, s0.acc: 89.8877

[>>>>>>>>>>>>>                ] 442/966, 1.4 task/s, elapsed: 309s, ETA:   367s

KeyboardInterrupt: 

### save config for test

In [20]:
import pickle
import os
with open(os.path.join(cfg.work_dir,'cfg.pkl'),'wb') as f:
    pickle.dump(cfg,f)